In [1]:
import cv2
import motors
import time
import numpy as np
from ultralytics import YOLO
import ipywidgets.widgets as widgets
from IPython.display import display
import traitlets
import pyzed.sl as sl
import threading
from traitlets.config.configurable import SingletonConfigurable

robot = motors.MotorsYukon(mecanum=False)
model = YOLO("yolo11l_half.engine", task='detect')

display_color = widgets.Image(format='jpeg', width='45%')
display_depth = widgets.Image(format='jpeg', width='45%')
layout = widgets.Layout(width='100%')
sidebyside = widgets.HBox([display_color, display_depth], layout=layout)
display(sidebyside)

def bgr8_to_jpeg(value):
    return bytes(cv2.imencode('.jpg', value)[1])

person_dropdown = widgets.Dropdown(options=[('None', None)], description='Select Person:')
tracking_status = widgets.Label(value="Status: No person selected")
display(widgets.VBox([person_dropdown, tracking_status]))

tracked_person_idx = None
def on_person_select(change):
    global tracked_person_idx
    tracked_person_idx = change['new']
person_dropdown.observe(on_person_select, names='value')

class Camera(SingletonConfigurable):
    color_value = traitlets.Any()
    depth_value = traitlets.Any()
    def __init__(self):
        super(Camera, self).__init__()
        self.zed = sl.Camera()
        p = sl.InitParameters()
        p.camera_resolution = sl.RESOLUTION.VGA
        p.depth_mode = sl.DEPTH_MODE.ULTRA
        p.coordinate_units = sl.UNIT.MILLIMETER
        if self.zed.open(p) != sl.ERROR_CODE.SUCCESS:
            self.zed.close()
            exit(1)
        self.runtime = sl.RuntimeParameters()
        self.thread_running_flag = False
        info = self.zed.get_camera_information()
        w = info.camera_configuration.resolution.width
        h = info.camera_configuration.resolution.height
        self.image = sl.Mat(w, h, sl.MAT_TYPE.U8_C4, sl.MEM.CPU)
        self.depth = sl.Mat(w, h, sl.MAT_TYPE.F32_C1, sl.MEM.CPU)
    def _capture_frames(self):
        while self.thread_running_flag:
            if self.zed.grab(self.runtime) == sl.ERROR_CODE.SUCCESS:
                self.zed.retrieve_image(self.image, sl.VIEW.LEFT)
                self.zed.retrieve_measure(self.depth, sl.MEASURE.DEPTH)
                bgra = self.image.get_data()
                self.color_value = cv2.cvtColor(bgra, cv2.COLOR_BGRA2BGR)
                self.depth_value = np.asanyarray(self.depth.get_data())
    def start(self):
        if not self.thread_running_flag:
            self.thread_running_flag = True
            threading.Thread(target=self._capture_frames, daemon=True).start()
    def stop(self):
        self.thread_running_flag = False

camera = Camera()
camera.start()

def execute_movement(turn, move):
    if move=='forward': robot.forward(0.7)
    elif move=='backward': robot.backward(0.7)
    else: robot.stop()
    if turn=='left': robot.left(0.4)
    elif turn=='right': robot.right(0.4)

def getPersonDepth(bbox, depth_image):
    cx = int((bbox[0]+bbox[2])/2)
    cy = int((bbox[1]+bbox[3])/2)
    d = depth_image[cy,cx]
    return d if not (np.isnan(d) or np.isinf(d)) else 0

def generate_robot_commands(bbox, fw, dist, desired_distance=1000):
    cx = (bbox[0]+bbox[2])/2
    ex = cx - fw/2
    if dist < desired_distance-200: move='backward'
    elif dist > desired_distance+200: move='forward'
    else: move='stop'
    turn = 'right' if ex>fw*0.1 else 'left' if ex< -fw*0.1 else 'none'
    return turn, move

def process_frame(frame, depth_frame):
    r = model(frame, verbose=False)[0]
    persons = [r.boxes.xyxy[i].cpu().numpy()
               for i in range(len(r.boxes.cls))
               if r.boxes.cls[i]==0 and r.boxes.conf[i]>0.5]
    opts = [('None', None)] + [(f"Person {i+1}", i) for i in range(len(persons))]
    if opts != person_dropdown.options:
        person_dropdown.options = opts
    if tracked_person_idx is not None and tracked_person_idx < len(persons):
        b = persons[tracked_person_idx]
        dist = getPersonDepth(b, depth_frame)
        turn, move = generate_robot_commands(b, frame.shape[1], dist)
        execute_movement(turn, move)
        tracking_status.value = f"Status: Tracking Person {tracked_person_idx+1} | Dist: {dist:.0f} mm"
        for i, bb in enumerate(persons):
            c = (0,255,0) if i==tracked_person_idx else (255,0,0)
            cv2.rectangle(frame, (int(bb[0]),int(bb[1])), (int(bb[2]),int(bb[3])), c, 2)
            if i!=tracked_person_idx:
                cv2.putText(frame, f"Person {i+1}", (int(bb[0]),int(bb[1]-10)),
                            cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,0),1)
        cv2.putText(frame, f"{move} {turn}", (10,30), cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,255,0),2)
    else:
        robot.stop()
        tracking_status.value = ("Status: No person selected"
                                 if tracked_person_idx is None
                                 else "Status: Selected person not found")
        for i, bb in enumerate(persons):
            cv2.rectangle(frame, (int(bb[0]),int(bb[1])), (int(bb[2]),int(bb[3])), (255,0,0),2)
            cv2.putText(frame,f"Person {i+1}",(int(bb[0]),int(bb[1]-10)),
                        cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,0),1)
    return frame


[2025-05-07 11:59:05 UTC][ZED][INFO] Logging level INFO
[2025-05-07 11:59:05 UTC][ZED][INFO] Logging level INFO
[2025-05-07 11:59:05 UTC][ZED][INFO] Logging level INFO
[2025-05-07 11:59:06 UTC][ZED][INFO] [Init]  Depth mode: ULTRA
[2025-05-07 11:59:06 UTC][ZED][INFO] [Init]  Camera successfully opened.
[2025-05-07 11:59:06 UTC][ZED][INFO] [Init]  Camera FW version: 1523
[2025-05-07 11:59:06 UTC][ZED][INFO] [Init]  Video mode: VGA@100
[2025-05-07 11:59:06 UTC][ZED][INFO] [Init]  Serial Number: S/N 39784002


In [2]:
import threading
import time

running = True
def run_loop():
    while running:
        if camera.color_value is not None and camera.depth_value is not None:
            f = camera.color_value.copy()
            d = camera.depth_value.copy()
            pf = process_frame(f, d)
            rc = cv2.resize(pf, None, fx=0.3, fy=0.3, interpolation=cv2.INTER_AREA)
            display_color.value = bgr8_to_jpeg(rc)
            dm = cv2.applyColorMap(cv2.convertScaleAbs(d, alpha=0.03), cv2.COLORMAP_JET)
            rd = cv2.resize(dm, None, fx=0.3, fy=0.3, interpolation=cv2.INTER_AREA)
            display_depth.value = bgr8_to_jpeg(rd)
        time.sleep(0.05)

threading.Thread(target=run_loop, daemon=True).start()


Loading yolo11l_half.engine for TensorRT inference...
[05/07/2025-12:59:38] [TRT] [I] Loaded engine size: 52 MiB
[05/07/2025-12:59:38] [TRT] [W] Using an engine plan file across different models of devices is not recommended and is likely to affect performance or even cause errors.
[05/07/2025-12:59:38] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +1, GPU +36, now: CPU 1, GPU 84 (MiB)
